# Combining samples

Currently, the data has individual folders and files for each sample. To put all of the sequencing information from various patients into a single file, the below code can be used.

## miRNA-seq

For each set of miRNA expression data, I can create a summary csv with the expression of each mature miRNA.

The difficulty comes in dealing with cross mapped miRNAs... There is no easy way to determine which other miRNA cross mapped miRNAs map to. I decided to just go with what the original <a  href="https://academic.oup.com/nar/article/44/1/e3/2499678">paper</a>/data analysis did, which is count the read for both miRNAs.

The summary file columns are the case id followed by the sample type. This sample type is a two digit code which 01-09 for tumors, 10-19 for normal and 20-29 for control (according to the outdated documentation <a href="https://wiki.nci.nih.gov/display/TCGA/TCGA+barcode">here</a>). A full list of codes is <a href="https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/sample-type-codes">here</a>.

In [ ]:
import json
import pandas as pd

def collect_miRNA(file_loc, meta_file, output_raw, output_rpm):
    """
        Collects all of miRNA-seq data from a TCGA project into two csv files, one with raw 
        and one with RPM normalized read counts
        
        The case IDs will be used as columns and the sample_type_id will be appended to the case ID
        01: Primary Solid Tumor
        02: Recurrent Solid Tumor
        05: Additional - New Primary
        06: Metastatic
        07: Additional Metastatic
        11: Solid Tissue Normal
    """
    # dictionaries which will be turned into dataframes at the end
    dict_raw = {}
    dict_rpm = {}
    # find the sample ids associated with each file and loop over the files
    meta_loc = "{}/{}".format(file_loc, meta_file)
    with open(meta_loc) as meta:    
        data = json.load(meta)
        try:
            # for metadata downloaded using the API
            data = data["data"]["hits"]
        except TypeError:
            # for metadata manually downloaded
            pass
        for f_info in data:
            f_name = f_info["file_name"]
            file_id = f_info["file_id"]
            case_id = f_info["associated_entities"][0]["case_id"]
            samps = f_info["cases"][0]["samples"]
            # check if multiple samples associated with the file
            if len(samps) != 1:
                print "Multiple samples associated with file {}".format(f_name)
            samp_type_id = samps[0]["sample_type_id"]
            
            # read the isoform data
            df = pd.read_table("{}/{}/{}".format(file_loc, file_id, f_name), header=0)
            # sum based on the miRNA region ie 'mature,MIMAT0000062'
            mat_df = df.groupby("miRNA_region").sum()
            # filter out non mature reads
            mat_df = mat_df[mat_df.index.str.contains("mature")]
            # remove mature annotation from index
            mat_df.index = mat_df.index.str.replace("mature,", "")
            col_name = "{}_{}".format(case_id, samp_type_id)
            dict_raw[col_name] = mat_df["read_count"]
            dict_rpm[col_name] = mat_df["reads_per_million_miRNA_mapped"]
    
    df_raw = pd.DataFrame(dict_raw)
    df_rpm = pd.DataFrame(dict_rpm)
    
    # fill any missing read counts with 0
    df_raw.fillna(0, inplace=True)
    df_rpm.fillna(0, inplace=True)
    
    df_raw.to_csv(output_raw)
    df_rpm.to_csv(output_rpm)

The below code can be used to create miRNA summary files for all downloaded TCGA projects.

In [ ]:
import os

def tcga_mirna_summary(tcga_dir):
    for in_dir in os.listdir(tcga_dir):
        print "Processing {}".format(in_dir)
        mir_dir = "{}/{}/miRNA-seq/Isoform Expression Quantification".format(tcga_dir, in_dir)
        # find json metadata file
        meta_file = [meta for meta in os.listdir(mir_dir) if "json" in meta][0]
        collect_miRNA(mir_dir, meta_file, 
                      "TCGA Summary/miRNA-seq/{} miRNA-seq Raw Reads.csv".format(in_dir),
                      "TCGA Summary/miRNA-seq/{} miRNA-seq RPM Reads.csv".format(in_dir))

## mRNA-seq

The same thing can be done for mRNA-seq information. The large number of samples in some of the datasets, such as BRAC, require large amounts of memory.

In [ ]:
import json
import pandas as pd

def collect_mRNA(file_loc, meta_file, output_file):
    """
        Collects all of mRNA-seq data from a TCGA project into a csv file
        
        The case IDs will be used as columns and the sample_type_id will be appended to the case ID
        01: Primary Solid Tumor
        02: Recurrent Solid Tumor
        05: Additional - New Primary
        06: Metastatic
        07: Additional Metastatic
        11: Solid Tissue Normal
    """
    # dictionary which will be turned into dataframe at the end
    dict_mrna = {}
    # find the sample ids associated with each file and loop over the files
    meta_loc = "{}/{}".format(file_loc, meta_file)
    with open(meta_loc, "r") as meta, open(output_file, "w") as fout:    
        data = json.load(meta)
        try:
            # for metadata downloaded using the API
            data = data["data"]["hits"]
        except TypeError:
            # for metadata manually downloaded
            pass
        for f_info in data:
            f_name = f_info["file_name"]
            file_id = f_info["file_id"]
            case_id = f_info["associated_entities"][0]["case_id"]
            samps = f_info["cases"][0]["samples"]
            # check if multiple samples associated with the file
            if len(samps) != 1:
                print "Multiple samples associated with file {}".format(f_name)
            samp_type_id = samps[0]["sample_type_id"]
            
            # read the HTSeq count data, the squeeze boolean ensures it will produce a series
            ser = pd.read_table("{}/{}/{}".format(file_loc, file_id, f_name), header=None, index_col=0, squeeze=True)
            # drop the extra rows from HTSeq count
            ser.drop(["__no_feature", "__ambiguous", "__too_low_aQual", "__not_aligned", "__alignment_not_unique"], inplace=True)
            
            col_name = "{}_{}".format(case_id, samp_type_id)
            dict_mrna[col_name] = ser
    
    df = pd.DataFrame(dict_mrna)
    df.to_csv(output_file)

These reads can then be TMM normalized (<a href="https://doi.org/10.1186/gb-2010-11-3-r25">TMM paper</a>). The data_processing package includes tmm_norm which is my implementation of EdgeR's <a href="https://github.com/Bioconductor-mirror/edgeR/blob/master/R/calcNormFactors.R">calcNormFactors</a>.

In [ ]:
import random
import pandas as pd
import data_processing as dp

def make_tmm(file_name):
    df = pd.read_csv(file_name, header=0, index_col=0)
    rand_samp = random.choice(df.columns.tolist())
    df_tmm = dp.tmm_norm(df, rand_samp)
    return df_tmm

The below code can be used to create mRNA summary files for all downloaded TCGA projects.

In [ ]:
import os

def tcga_mrna_summary(tcga_dir):
    for in_dir in os.listdir(tcga_dir):
        print "Processing {}".format(in_dir)
        mrna_dir = "{}/{}/mRNA-seq".format(tcga_dir, in_dir)
        # find json metadata file
        meta_file = [meta for meta in os.listdir(mrna_dir) if "json" in meta][0]
        out_file = "{}/TCGA Summary/mRNA-seq/{} mRNA-seq Reads.csv".format(tcga_dir, in_dir)
        # check if file already exists
        if os.path.isfile(out_file):
            continue
        collect_mRNA(mrna_dir, meta_file, out_file)
        tmm_df = make_tmm(out_file)
        # filtered to remove genes for which there is not at least 1 TMM normalized read in at least one sample
        # also truncate to one decimal place to help limit file size
        tmm_filt = tmm_df[tmm_df.applymap(lambda x: x >= 1).any(axis=1)]
        tmm_filt.to_csv("D:/TCGA/TCGA Summary\mRNA-seq\{} mRNA-seq Reads (TMM Norm).csv".format(in_dir), float_format="%.1f")